## Volume Demand Forecast using DeepAR: Making Predictions with Deployed Model as an Endpoint

### Cold Start Forecasting: Making prediction for a new agency and sku combination
DeepAR can generate forecasts for new time series that are similar to the ones it has been trained on. In our case, each agency sells a number of SKUs that might be different from each other. For example, Agency_01 sells SKU 01, 02, 03, 04, 05 and 11 but what if we wonder how well agency_01 can sell SKU_07? So, we predicted a new agency and sku combination,Agency_01 and SKU_07, for year 2017 below. Since some other agencies sell SKU_07 in 2017 so we chose dydamic features, sales and promotions, to be the median value of these agencies. Also,the monthly predictions of volume demand for Agency_01-SKU_07 in 2017 shown below includes predictions of average of 100 predictions, 0.9 and 0.5 quantiles.

In [3]:
#predict a new agency_01, sku_07 : [0,6]
import json
import pandas as pd
request_Agency2_sku7 = json.dumps(
{
    "instances":[
    
        {
            'start':str(pd.Timestamp("2017-01-01 00:00:00")),
            'target':[],
            'cat':[1,6],
            'dynamic_feat':[[1321,1464,1432,1422,1422,1443,1432,1432,1403,1466,1428,1426], #median sales
                            [20,47,117,128,146,104,120,148,174,99,91,114]] #median promotions
        }
    ],
    
    "configuration":{'num_samples':100, 'output_types':['mean', 'quantiles'],'quantiles': ['0.5', '0.9']}
    
}
) 
endpoint_name = 'predict-volume-deepar-endpoint'


import boto3
client = boto3.client('sagemaker-runtime')
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=request_Agency2_sku7)

content = response['Body'].read().decode('utf-8')
content

'{"predictions":[{"quantiles":{"0.9":[1890.9265136719,1867.8132324219,2033.0036621094,2107.3288574219,2177.2241210938,2205.8798828125,2208.4426269531,2068.5144042969,2149.599609375,2171.4392089844,2233.701171875,2210.1220703125],"0.5":[830.7008666992,1373.9489746094,1429.7987060547,1508.3709716797,1562.9676513672,1594.1791992188,1641.9381103516,1528.3635253906,1578.1357421875,1535.7589111328,1467.7604980469,1542.6149902344]},"mean":[821.3439331055,1352.5205078125,1439.2410888672,1526.8978271484,1582.6636962891,1608.3604736328,1620.4918212891,1512.2180175781,1582.4641113281,1533.2131347656,1486.3603515625,1524.2233886719]}]}'